In [4]:
import streamlit as st
import sqlalchemy as sq
import pandas as pd
import datetime as dt

In [30]:
engine = sq.create_engine("sqlite:///find_me.db")
cnx = engine.connect()

In [31]:
df = pd.DataFrame(data={
    "current_loc":["current city"],
    "current_date":[dt.datetime.now().date()],
    "future_loc":["new city"],
    "future_date":[dt.datetime.strptime("02/28/2021","%m/%d/%Y")]
})

In [32]:
df.to_sql("location",con=cnx,if_exists="replace",index=False)

In [19]:
meta = sq.MetaData()
meta.reflect(bind = engine)
table = meta.tables['location']

In [20]:
meta = sq.MetaData()
meta.reflect(bind = engine)
table = meta.tables['location']
query = sq.insert(table).values(current_loc = current_loc, 
                               current_date = dt.datetime.now().date(),
                               future_loc = future_loc,
                               future_date = future_date
                              )

Table('location', MetaData(bind=None), Column('index', BIGINT(), table=<location>), Column('current_loc', TEXT(), table=<location>), Column('current_date', DATE(), table=<location>), Column('future_loc', TEXT(), table=<location>), Column('future_date', DATETIME(), table=<location>), schema=None)

In [12]:
df.to_sql("location",con=cnx)

In [15]:
pd.read_sql("location",con=cnx)

,index,current_loc,current_date,future_loc,future_date
0,0,mexico city,2021-02-21,"naples, fl",2021-02-28


# Walk Score API

In [186]:
import requests as r
def get_walkscore(lat, lon, address):
    api_key = "18462ed72359fe80c186f0796f512165"
    url = "https://api.walkscore.com/score?format=json"
    addie = address.replace(" ", "%20")
    params = {
        'address':addie,
        'lat':lat,
        'lon':lon,
        'wsapikey':api_key
    }
    response = r.get(url=url,params=params)
    return response

In [187]:
response = get_walkscore(address='1119 8th Avenue Seattle WA 98101',lat='47.6085',lon='-122.3295')

In [188]:
import pandas as pd

In [248]:
df = pd.read_html('data/walkscore.html')[0]

In [242]:
df

,City,State,Walk Score,Transit Score,Bike Score,Population
0,New York,NY,88.3,84.3,70.0,8175133
1,San Francisco,CA,87.4,80.3,72.2,805235
2,Jersey City,NJ,86.8,70.6,70.3,247597
3,Boston,MA,82.0,72.5,70.5,617594
4,Philadelphia,PA,78.8,66.8,67.4,1526006
...,...,...,...,...,...,...
103,Montgomery,AL,26.1,16.4,38.3,205764
104,Jacksonville,FL,25.7,23.0,42.7,821784
105,Winston-Salem,NC,23.4,--,29.2,229617
106,Fayetteville,NC,21.4,--,33.2,200564


# Weather API

In [191]:
import requests as r
import numpy as np
import datetime as dt

class weatherAnalysis():
    def __init__(self, location, type_='future'):
        '''
        Initiates weather api
        
        input
        ----
        location: str
            Zip code, ip address, lat/lon, city name
        type_: str
            current or future or history
        '''
        api_key = "4216ca74f3d1478dbbd154714212001"

        params = {
            'key':api_key,
            'q':location
        }

        if type_ == 'current':
            url = 'https://api.weatherapi.com/v1/current.json'
        elif type_ == 'future':
            url = 'https://api.weatherapi.com/v1/forecast.json'
            params['days'] = 3 # max forecast for free account
        elif type_ == 'history':
            url = 'https://api.weatherapi.com/v1/history.json'
            self.history(url, params)
            return
            
        response = r.get(url = url, params = params)
        self.response = response.json()

        if type_ == 'future':
            self.forecast()
        
    def date_lister(self):
        '''
        Returns list of 7 properly formatted dates
        '''
        historic_dates = []
        
        for i in range(1,8):
            new_date = dt.datetime.now() - dt.timedelta(days=i) # max history for free account
            historic_dates.append(new_date.strftime('%Y-%m-%d'))
        
        return historic_dates
    
    def forecast(self):
        '''
        Gets the forecast, calculates mean temp
        '''
        temps = []
        all_forecast = self.response['forecast']['forecastday']
        for day in all_forecast:
            temp = day['day']['avgtemp_f'] # get avg temp for day
            temps.append(temp)
        
        self.all_temps = temps
        self.mean_temp = np.mean(temps)
        
    def history(self, url, params):
        '''
        Gets historic weather, calculates mean temp
        
        input
        -----
        url: str
            Api url
        params: dict
            Constructed params, without date
            
        output
        ------
        self.historic: pd.DataFrame
            Dataframe of dates and temp in F
        '''
        temps = []
        date_list = self.date_lister()
        
        all_responses = []
        for date in date_list:
            params['dt'] = date
            response = r.get(url=url, params=params)
            all_responses.append(response.json())
        
        extracted = {}
        for i in range(len(all_responses)):
            selected = all_responses[i]['forecast']['forecastday'][0] # all info for selected day
            day = selected['day'] # temp stats for the day    
            extracted[selected['date']] = [day['avgtemp_f'],day['mintemp_f'],day['maxtemp_f']]
            
        df = pd.DataFrame(extracted).T.reset_index()
        df.columns = ['date','avg_temp_f','min_temp_f','max_temp_f']
        self.historic = df 

In [195]:
w = weatherAnalysis('San Francisco', type_='history')

In [196]:
w.historic.avg_temp_f.mean()

59.75714285714286

In [222]:
weather_avg = {}
weather_dfs = {}

In [243]:
new_df = pd.DataFrame(columns=list(df.columns) + ['temp'])

In [244]:
new_df

,City,State,Walk Score,Transit Score,Bike Score,Population,temp


In [247]:
cities = ['New York','Jersey City', 'San Francisco']

In [223]:
for city in cities:
    try:
        w = weatherAnalysis(city, type_='history')
        w_df = w.historic
        weather_avg[city] = w_df.avg_temp_f.mean()
        weather_dfs[city] = w_df
    except:
        pass

In [255]:
locs = []
for i, r in df.iterrows():
    for x in cities:
        if x in r['City']:
            locs.append(i)

In [259]:
small_df = df.loc[locs]
small_df['temp'] = small_df['City'].map(weather_avg)

In [260]:
small_df

,City,State,Walk Score,Transit Score,Bike Score,Population,temp
0,New York,NY,88.3,84.3,70.0,8175133,40.357143
1,San Francisco,CA,87.4,80.3,72.2,805235,59.757143
2,Jersey City,NJ,86.8,70.6,70.3,247597,40.157143
